# 1. Zbiór danych
<p>
Zbiór danych dotyczący zdatności wody do picia. Zawiera 20 cech, które przedstawiają zawartość poszczególnych związków chemicznych, pierwiastków i mikroorganizmów oraz cechę określającą zdatność do spożycia.
</p>
https://www.kaggle.com/mssmartypants/water-quality

## Wczytanie zbioru

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt

data = pd.read_csv('datasets/waterQuality1.csv', delimiter=',')
data = data[data.is_safe != '#NUM!']
data = data[data.ammonia != '#NUM!']
data['ammonia'] = pd.to_numeric(data['ammonia'])
data.head()

## Usuwanie losowych wartości

In [ ]:
for index, row in data.iterrows():
    if index % 10 == 0:
        random_col = np.random.choice(data.columns)
        data.at[index, random_col] = np.nan

## Zastąpienie brakujących wartości średnią

In [ ]:
means = data.mean(axis=0)
for index, value in means.items():
    data[index].fillna(value=value, inplace=True)

## Zastąpienie brakujących wartości medianą

In [ ]:
medians = data.median(axis=0)
for index, value in medians.items():
    data[index].fillna(value=value, inplace=True)

## Zastąpienie najczęściej występujacą wartością

In [ ]:
value = data['is_safe'].value_counts().idxmax()
data['is_safe'].fillna(value=value, inplace=True)

# 2. Statystyki opisowe i podsumowujące

In [ ]:
data.describe()

In [ ]:
data.info()

## Zależności między zmiennymi
Scatter ploty między każdą parą zmiennych. Na przekątnej wykres gęstości prawdopodobieństwa (rozkład) zmiennej.

In [ ]:
import seaborn as sns

sns.pairplot(data)
plt.show()

#### Zmiennej podejrzane o relacje

In [ ]:
sns.pairplot(data[['bacteria', 'viruses']])
plt.show()

## Tabela korelacji

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(16, 9), dpi=80)

corrMatrix = data.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

## Histogramy

In [ ]:
print(data.columns)

data.hist(figsize=(30, 30))
plt.show()


Zmiennej podejrzane o relacje
In [5]:
sns.pairplot(data[['bacteria', 'viruses']])
plt.show()
￼## Boxploty

In [ ]:
data.boxplot(figsize=(30, 30))
plt.show()

#### Odfiltrowanie wartości odstających

In [ ]:
from scipy import stats
filtered = data[(np.abs(stats.zscore(data['aluminium'])) < 3)]
filtered = filtered[(np.abs(stats.zscore(filtered['arsenic'])) < 3)]
filtered = filtered[(np.abs(stats.zscore(filtered['nitrites'])) < 3)]

filtered.boxplot(figsize=(30, 30))
plt.show()
filtered.describe()

In [ ]:
filtered.info()

# 3. Skalowanie cech

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(data[['aluminium', 'ammonia', 'arsenic', 'barium', 'cadmium', 'chloramine', 'chromium', 'copper', 'flouride', 'bacteria', 'viruses', 'lead', 'nitrates', 'nitrites', 'mercury', 'perchlorate', 'radium', 'selenium', 'silver', 'uranium']])
data_scaled = scaler.transform(filtered[['aluminium', 'ammonia', 'arsenic', 'barium', 'cadmium', 'chloramine', 'chromium', 'copper', 'flouride', 'bacteria', 'viruses', 'lead', 'nitrates', 'nitrites', 'mercury', 'perchlorate', 'radium', 'selenium', 'silver', 'uranium']])

data_scaled = pd.DataFrame(np.append(data_scaled, filtered[['is_safe']].to_numpy(), axis=1), dtype=float)
data_scaled.columns = data.columns

data_scaled.boxplot(figsize=(30, 9))
plt.show()
data_scaled.describe()

In [ ]:
data.info()

# 4. Redukcja wymiarowości

### Algorytm PCA

In [ ]:
from sklearn.decomposition import PCA

x = data_scaled.loc[:, data_scaled.columns[:-1]].values
y = data_scaled.loc[:,['is_safe']].values

pca = PCA(n_components=16)
pca.fit(x)
principalComponents = pca.transform(x)
print(pca.explained_variance_ratio_.sum())